In [ ]:
# import libraries
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
#from sklearn.neighbors import KNeighborsClassifier #This one needs two arguments (x,y)
from sklearn.neighbors import NearestNeighbors #Just one argumente (pivoted dataframe)
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-04-11 01:33:53--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  19.7MB/s    in 1.3s    

2024-04-11 01:33:54 (19.7 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
#check for any null/empty values
print("\nempty values books:",df_books.isnull().any().sum(), "\nempty values ratings", df_ratings.isnull().any().sum())


empty values books: 1 
empty values ratings 0


In [ ]:
#Cleaning of null values
df_books.dropna(inplace=True)

In [ ]:
df_ratings.shape

(1149780, 3)

In [ ]:
#Filtering users with less than 200 ratings
user_ratings = df_ratings['user'].value_counts()
filter1 = user_ratings < 200  #Rule needed

filtered_df = df_ratings[~df_ratings['user'].isin(user_ratings[filter1].index)]


In [ ]:
#Filtering books with less than 100 ratings
book_ratings = df_ratings['isbn'].value_counts()
filter2 = book_ratings < 100  #Rule needed

filtered_df = filtered_df[~filtered_df['isbn'].isin(book_ratings[filter2].index)]

In [ ]:
filtered_df.shape

(49781, 3)

In [ ]:
filtered_df.head()

,user,isbn,rating
1456,277427,002542730X,10.0
1469,277427,0060930535,0.0
1471,277427,0060934417,0.0
1474,277427,0061009059,9.0
1484,277427,0140067477,0.0


In [ ]:
# Create Item-user matrix using pivot_table()  (FEATURE MATRIX)
df = filtered_df.pivot_table(index=['user'],columns=['isbn'],values='rating',fill_value=0).transpose()  # Transpose for user-centric view

df.head()

user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
isbn,,,,,,,,,,,,,,,,,,,,,
002542730X,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,10,0,0,0
0060008032,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0060096195,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
006016848X,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0060173289,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
print(user_ratings.dtypes)
print(df_books[['isbn', 'title']].dtypes)


user              
254     0060930535    0.0
        0061020710    0.0
        014028009X    0.0
        0140298479    0.0
        014100018X    8.0
                     ... 
278418  1878424319    NaN
        0553292722    NaN
        0553571818    NaN
        0786889020    0.0
        0399149325    NaN
Name: isbn, Length: 649128, dtype: float64
isbn     object
title    object
dtype: object


In [ ]:
# Modifying view a little
df.index = df.join(df_books.set_index('isbn'), on='isbn')['title']  # Join with books DataFrame and set index to 'title'

df.head()

user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
Politically Correct Bedtime Stories: Modern Tales for Our Life and Times,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,10,0,0,0
Angels,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
The Boy Next Door,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Men Are from Mars, Women Are from Venus: A Practical Guide for Improving Communication and Getting What You Want in Your Relationships",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Divine Secrets of the Ya-Ya Sisterhood : A Novel,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df.loc["2nd Chance"][:5]


user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
2nd Chance,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2nd Chance,0,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#KNN MODEL
k = 5
knn = NearestNeighbors(metric='cosine', n_neighbors=k)
knn.fit(df)

NearestNeighbors(metric='cosine')

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  recc = df.query("index == @book")
  # Find nearest neighbors (excluding the user themself)
  distances, neighbors = knn.kneighbors(recc.values.reshape(1, -1))

  recommended_books = []

  # Include queried book title
  recommended_books.append(book)
  # Loop through top k neighbors (excluding the first element for queried book)
  recommendations = []
  for i in range(1, min(k, len(neighbors.flatten()))):
    neighbor_index = neighbors.flatten()[i]
    neighbor_title = df.index[neighbor_index]
    neighbor_distance = distances.flatten()[i]
    recommendations.append([neighbor_title, neighbor_distance])

  # Add recommendations list to main list
  recommended_books.append(recommendations)
  recommended_books[1] = sorted(recommended_books[1], key=lambda x: x[1], reverse=True) # Sort the nested list based on the second element (distance) in descending order
  recommended_books = recommended_books[:5] #First five elements only

  return recommended_books

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Weight of Water', 0.7708583572697412], ['The Surgeon', 0.7699410973804288], ['I Know This Much Is True', 0.7677075092617776], ['The Lovely Bones: A Novel', 0.7234864549790632]]]


In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Weight of Water', 0.7708583572697412], ['The Surgeon', 0.7699410973804288], ['I Know This Much Is True', 0.7677075092617776], ['The Lovely Bones: A Novel', 0.7234864549790632]]]
You passed the challenge! 🎉🎉🎉🎉🎉
